<a href="https://colab.research.google.com/github/nbadino/ItaOil/blob/main/oil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

prova